**DEATH PREDICT**

*Predicting heart failure in hospital ICU (Intensive car Unit)*

In [64]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [65]:
# Read CSV file into DataFrame df
df = pd.read_csv('/kaggle/input/in-hospital-mortality-prediction/data01.csv', index_col=0)
# Show dataframe
df

In [66]:
df.describe()

In [67]:
df.info()

In [68]:
df.dropna(inplace=True)
df.info

In [69]:
# Average Age of non-surviving patients 
deads = df[df.outcome == 1]
print(np.mean(deads.age))

In [70]:
import seaborn as sns
corr = df.corr()
sns.heatmap(corr,
           xticklabels=corr.columns.values,
           yticklabels=corr.columns.values)

In [71]:
# Visualization of Heart Failures
'''
fig, ax = plt.subplots()
dead_count = df['outcome'].value_counts() 
dead_count.plot(ax=ax, kind='bar')
'''

In [72]:
# Visualization of Heart Failures

plt.figure(figsize=(16,8))
plt.title('Frequency of Heart Failures')
plt.hist(df['outcome'])
plt.xlabel('Heart Failure (0 = No) (1 = Yes)', fontsize=18)
plt.ylabel('Number of patients', fontsize=18)
plt.show()

In [73]:
# create the labels (ground truth) and features (data to train)
labels = df['outcome']
features = df.iloc[:, 2:]

X = features

y = np.ravel(labels)

In [74]:
# creating train and test data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = 0.33, random_state = 42)

In [75]:
# scale the data ( convert value between -1 and 1)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)

X_test = scaler.transform(X_test)  

X_train

In [76]:
X_train.shape

In [77]:
# build the model
nb_col = X_train.shape[1]

model = Sequential()
model.add(Dense(nb_col, activation='relu' , input_shape=(nb_col, )))
model.add(Dense(nb_col, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [78]:
# compile the model
model.compile(optimizer='sgd', loss='binary_crossentropy')

In [79]:
#train the model
model.fit(X_train, y_train, batch_size=1, epochs=1, verbose=1)

In [80]:
# get the models predicted outcome (death)
y_predictions = model.predict_classes(X_test)

In [81]:
score = model.evaluate(X_test, y_test, verbose=1)
score